In [2]:
#TODO
#1. Add the 64*64 outputs to one image.
#2. For every pixel, convert it into the most probable one.Maybe depend on neighbours??
# Could be multiple overlaps, find a way to handle that.
#3. Convert every pixel to correct polygon json format

In [9]:
import numpy as np
import cv2
import json
import satlaspretrain_models

In [4]:
def post_process_optimized(outputs, gamma=0.5):
    empty_img = np.zeros((1024, 1024, 5))
    processed_img = np.zeros((1024, 1024), dtype=int)
    
    for x, y, probs in outputs:
        empty_img[x:x+256, y:y+256] += np.array(probs)
    
    padded = np.pad(empty_img, ((1,1), (1,1), (0,0)))
    neighbors = gamma * (padded[:-2, 1:-1] + padded[2:, 1:-1] + 
                        padded[1:-1, :-2] + padded[1:-1, 2:])
    
    processed_img = np.argmax(empty_img + neighbors, axis=2)
    return processed_img

def test_post_process_optimized():
    positions = np.mgrid[0:1024:256, 0:1024:256].reshape(2, -1).T
    
    probs = np.random.dirichlet(np.ones(5)*2, size=positions.shape[0])
    probs[:,0] = np.clip(probs[:,0], 0.3, 1.0)
    probs = probs / probs.sum(axis=1, keepdims=True)
    
    outputs = [(x, y, p.tolist()) for (x, y), p in zip(positions, probs)]
    return post_process_optimized(outputs)

In [15]:
test_post_process_optimized()

array([[2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 4]])

## torch and cuda version

In [5]:
import torch

def post_process_torch(outputs, gamma=0.5):
    device = torch.device("cuda")  # Assuming model outputs are on CUDA
    empty_img = torch.zeros((1024, 1024, 5), device=device)
    
    # Convert outputs to tensor once
    coords = torch.tensor([[x,y] for x,y,_ in outputs], device=device)
    probs = torch.tensor([p for _,_,p in outputs], device=device)
    
    # Batch assignment
    for coord, prob in zip(coords, probs):
        empty_img[coord[0]:coord[0]+64, coord[1]:coord[1]+64] += prob
    
    padded = torch.nn.functional.pad(empty_img, (0,0,1,1,1,1))
    neighbors = gamma * (padded[:-2, 1:-1] + padded[2:, 1:-1] + 
                        padded[1:-1, :-2] + padded[1:-1, 2:])
    
    return torch.argmax(empty_img + neighbors, dim=2)  # Keep on GPU if needed

## From pixel img to polygon

In [6]:

# Example: A random 1024x1024 tensor with values 0-4
def converter(tensor):
# Dictionary to store polygons for each unique value
    polygons = {}

    for val in range(5):  # Iterate over unique values (0-4)
        mask = (tensor == val).astype(np.uint8)  # Create binary mask for the value

        # Find contours (polygons) of connected components
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Convert contours to a list of polygons
        polygons[val] = [c.reshape(-1, 2).tolist() for c in contours]
    return polygons
test=converter(test_post_process_optimized())


def write_json(polygons, filename:str):
    #Function that takes list of polygons and writes them to a json file.
    #The list of polygons is a list of lists of polygons, where each polygon is a list of points.
    # Dictionary to map unique values to class names
    numper_2_class={1:"plantation",2:"grassland_shrubland",3:"mining",4:"logging"}
    with open(f"{filename}.json", "a") as file:
        images_overview={"images":[]}
        # Iterate over images in polygons
        for i in range(len(polygons)):

            curr={"file_name":f"evaluation_{i}.tif","annotations":[]}
            #Iterate over types of polygons in image
            for j in range(1,len(polygons[i])):
                # Convert polygons to list of coordinates

                for k in range(len(polygons[i][j])):
                    listed_polygons=[]

                    for p in range(len(polygons[i][j][k])):
                        listed_polygons.append(polygons[i][j][k][p][0])
                        listed_polygons.append(polygons[i][j][k][p][1])
                    curr["annotations"].append({"class":numper_2_class[j],"segmentation":listed_polygons})
            images_overview["images"].append(curr)
        file.write(json.dumps(images_overview, ensure_ascii=False, indent=4))
write_json([test],"test")
# Now, polygons[val] contains a list of polygons for each unique value.
